<a href="https://colab.research.google.com/github/jack-cao-623/python_learning/blob/main/pandas_merge_join_concat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas merging, joining, and concatenating
- combine 2 or more dataframes
- in Python Pandas:
  - join is only based on index
  - merge is more flexible and can combine using index or columns

In [2]:
# libraries needed
import numpy as np
import pandas as pd

In [3]:
# datasets we'll need
week1_sales = pd.read_csv(
    'https://raw.githubusercontent.com/jack-cao-623/python_learning/main/pandas/Restaurant%20-%20Week%201%20Sales.csv'
)

week2_sales = pd.read_csv(
    'https://raw.githubusercontent.com/jack-cao-623/python_learning/main/pandas/Restaurant%20-%20Week%202%20Sales.csv'
)

customers = pd.read_csv(
    'https://raw.githubusercontent.com/jack-cao-623/python_learning/main/pandas/Restaurant%20-%20Customers.csv'
)

foods = pd.read_csv(
    'https://raw.githubusercontent.com/jack-cao-623/python_learning/main/pandas/Restaurant%20-%20Foods.csv'
)

In [4]:
# examine data

# week1_sales is one row per order: which customer ordered which food in week 1
week1_sales.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [5]:
# week2_sales is one row per order: which customer ordered which food in week 2
week2_sales.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [6]:
# one row per restaurant customer; 'ID' column matches to 'Customer ID' column in week1_sales and week2_sales
customers.head()

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive
3,4,Steven,Evans,Male,Fatz,Registered Nurse
4,5,Judy,Morrison,Female,Demivee,Legal Assistant


In [7]:
# foods is one row per food
foods.head()

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99
3,4,Quesadilla,4.25
4,5,Pizza,2.49


## pd.concat() method
- stack dataframes on top of each other
- like SQL UNION ALL
- syntax is pd.concat([df1, df2], ignore_index = True or False)
  - ignore_index = False keeps the same indices
  - ignore_index = True resets the index, i.e., a fresh one
- alternative syntax is df1.append(df2, ignore_index = True or False)

In [8]:
# week1_sales and week2_sales have same structure; stack on top of each other
pd.concat(
    objs = [week1_sales, week2_sales]     # week1_sales on top; week2_sales below
)

# note that there are 500 rows, but index is from 0-249
# this is because there are duplicated index values: 0-249 for week1_sales and 0-249 for week2_sales

# if you want an index from 0-499:
  # ignore_index = True

# see below for how this works

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,783,10
246,556,10
247,547,9
248,252,9


In [9]:
fake_dat_a = pd.DataFrame(data = [1, 2, 3], index = [0, 1, 2])
fake_dat_b = pd.DataFrame(data = [4, 5, 6], index = ['a', 'b', 'c'])

pd.concat(
    objs = [fake_dat_a, fake_dat_b],
    ignore_index = False              # False by default, which preserves original index in each dataframe; True resets the index
)

,0
0,1
1,2
2,3
a,4
b,5
c,6


In [10]:
# new sales dataframe with combines week1_sales and week2_sales
pd.concat(
    objs = [week1_sales, week2_sales],
    ignore_index = True                    # re-index, i.e., a new index
)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


In [11]:
# above is equivalent to:
(
    week1_sales
      .append(week2_sales, ignore_index = True)
)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


In [12]:
# multi-index such that we have unique indices and we can identify whether row came from week1_sales or week2_sales
sales = pd.concat(
    objs = [week1_sales, week2_sales],
    keys = ['week1', 'week2']                  # don't need to add indicator column manually
)

sales

# keys argument must be same length as objects
# these labels are the outermost layer
# innermost layer are original indices

Customer ID  Food ID
week1 0            537        9
      1             97        4
      2            658        1
      3            202        2
      4            155        9
...                ...      ...
week2 245          783       10
      246          556       10
      247          547        9
      248          252        9
      249          249        6

[500 rows x 2 columns]

## Inner Joins using .merge() method

In [13]:
# customers who ordered in week 1 and week 2

In [14]:
week1_sales.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [15]:
week2_sales.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [16]:
# customers who ordered in week 1 and week 2
(
    week1_sales
      .merge(
          right = week2_sales,
          how = 'inner', 
          left_on = 'Customer ID',
          right_on = 'Customer ID'
      )
)

# Food ID_x refers to food ordered in week 1
# Food ID_y refers to food ordered in week 2
# Customer ID 155 is repeated
# Customer 155 ordered foods 9 and 1 in week 1; ordered food 3 in week 2

,Customer ID,Food ID_x,Food ID_y
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9
...,...,...,...
57,945,5,4
58,343,3,5
59,343,3,2
60,343,3,7


In [17]:
# customer 155 in week 1
week1_sales[week1_sales['Customer ID'] == 155]

,Customer ID,Food ID
4,155,9
17,155,1


In [18]:
# customer 155 in week 2
week2_sales[week2_sales['Customer ID'] == 155]

,Customer ID,Food ID
208,155,3


In [19]:
# see join in action for customer 155
(
week1_sales[week1_sales['Customer ID'] == 155]                     # left table
  .merge(
      right = week2_sales[week2_sales['Customer ID'] == 155],      # right table
      how = 'inner',                                               # inner join
      left_on = 'Customer ID',                                     # join on this key from left table
      right_on = 'Customer ID'                                     # join on this key from right table
  )
)

,Customer ID,Food ID_x,Food ID_y
0,155,9,3
1,155,1,3


In [20]:
# rename suffixes
(
    week1_sales                                  # left table
      .merge( 
          right = week1_sales,                   # right table
          how = 'inner',                         # inner join left and right tables
          left_on = 'Customer ID',               # key for left table
          right_on = 'Customer ID',              # key for right table
          suffixes = ['_week1', '_week2']        # suffixes; default is '_x' and '_y'
      )
)

,Customer ID,Food ID_week1,Food ID_week2
0,537,9,9
1,97,4,4
2,658,1,1
3,202,2,2
4,155,9,9
...,...,...,...
313,413,9,9
314,926,6,6
315,134,3,3
316,396,6,6


In [21]:
# customers who ordered in week 1 and week 2 and got the same thing each week
# inner join across multiple columns
(
    week1_sales                                       # left table
      .merge(       
          right = week2_sales,                        # right table
          how = 'inner',                              # inner join left and right tables
          left_on = ['Customer ID', 'Food ID'],       # join on Customer ID = Customer ID and Food ID = Food ID
          right_on = ['Customer ID', 'Food ID']
      )
)

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,21,4
6,922,1
7,578,5
8,578,5


In [22]:
# above is equivalent to
(
    week1_sales
      .merge(
          right = week2_sales,
          how = 'inner',
          left_on = 'Customer ID',
          right_on = 'Customer ID'
      )
      .query("`Food ID_x` == `Food ID_y`")
)

,Customer ID,Food ID_x,Food ID_y
16,304,3,3
24,540,3,3
26,937,10,10
28,233,3,3
29,21,4,4
30,21,4,4
37,922,1,1
54,578,5,5
55,578,5,5


In [23]:
# customer 578 is duplicated
week1_sales[week1_sales['Customer ID'] == 578]

,Customer ID,Food ID
224,578,5


In [24]:
week2_sales[week2_sales['Customer ID'] == 578]

# got food #5 once in week 1 and twice in week 2

,Customer ID,Food ID
29,578,5
189,578,5


In [25]:
# customers who only ordered in week 1 but not in week 2
week1_customers_only = (
    week1_sales                                                
      .merge(
          right = week2_sales,
          how = 'left',
          left_on = 'Customer ID',
          right_on = 'Customer ID'
      )
      .query('`Food ID_y`.isna()', engine = 'python')
      ['Customer ID']
      .unique()
)

week1_customers_only

array([  97,  658,  202,  213,  600,   71,  174,  961,  966,  641,  288,
        149,  954,  147,  549,   78,  514,  833,  329,  586,  341,  519,
        680,  419,   20,  822,  226,  203,  296,  821,  697,  264,  477,
        524,  121,  290,  100,  260,  896,  953,  682,  809,  450,  772,
        159,  876,  864,   68,  812,  921,  941,  108,  315,  358,  491,
        110,  737,  836,  749,  758,  527,  433,  250,  504,  910,  351,
        282,  117,   63,  144,  393,  380,  515,  357,    3,  875,  352,
         93,  323,   64,  912,  327,  399,  418,  669,  259,  410,  363,
        728,  244,  319,   51,   26,  472,  608,  160,  645,  374,  762,
        332,  338,  140,  567,  602,   10,   74, 1000,  881,  107,  703,
        225,  962,  114,  346,  191,  331,  738,  427,  902,  385,  555,
         67,  138,  775,  648,  475,  483,  313,  263,  871,  747,   38,
        190,  348,  167,  671,  501,  406,   47,  991,  539,   53,  493,
        650,  848,  307,  606,  985,  123,  764,   

In [26]:
# check
(
    week2_sales['Customer ID']        
      .isin(week1_customers_only)   # whether week 2 customers include any of the week 1 customers (True) or not (False)
      .sum()                        # sum up to get how many week 2 customers are week 1 cusotmers only; should be 0
)


0

In [27]:
# week 2 customers who weren't week 1 customers, what foods did they get?
(
    week2_sales                           # left table, x
      .merge(
          right = week1_sales,            # right table, y
          how = 'left',                   # keep all rows in left table, x
          left_on = 'Customer ID',        # join on Customer ID
          right_on = 'Customer ID'
      )
      .query('`Food ID_y`.isna()', engine = 'python')    # filter to customers who only ordered in week 2 and not in week 1
      #[['Customer ID', 'Food ID_x']]
      .merge(                              # get food items
          right = foods, 
          how = 'left', 
          left_on = 'Food ID_x',
          right_on = 'Food ID'
      )
      ['Food Item']                        # count how many times each food was ordered
      .value_counts()
)


Drink         30
Burrito       26
Pasta         23
Quesadilla    20
Salad         20
Donut         18
Steak         17
Sushi         16
Taco          15
Pizza         12
Name: Food Item, dtype: int64

In [28]:
# what was the most popular food item among customers who ordered both weeks?
(
    week1_sales
      .merge(
          right = week2_sales,
          how = 'inner', 
          left_on = 'Customer ID', 
          right_on = 'Customer ID',
          suffixes = ['_week1', '_week2']

      )
      .melt(
          id_vars = 'Customer ID',
          var_name = 'week', 
          value_name = 'food_id'
      )
      .merge(
          right = foods, 
          how = 'left', 
          left_on = 'food_id',
          right_on = 'Food ID'
      )
      .groupby(['week', 'Food Item'])
      [['Food Item']]
      .count()
      #.value_counts()
      #['food_id']
      #.value_counts()
      #.groupby(['week', 'food_id'])
      #.count()
      #.reset_index()
      #.rename(columns = {'Customer ID': 'num'})
      #.sort_values(by = ['week', 'num'], ascending = [True, False])
)

Food Item
week          Food Item            
Food ID_week1 Burrito             5
              Donut              10
              Drink               5
              Pasta               5
              Pizza               5
              Quesadilla          6
              Salad               2
              Steak               5
              Sushi              10
              Taco                9
Food ID_week2 Burrito             5
              Donut               2
              Drink               5
              Pasta               4
              Pizza              10
              Quesadilla          9
              Salad               5
              Steak               9
              Sushi               3
              Taco               10

## Outer Joins
- Full Outer Joins, not Left and Right Joins

In [29]:
# customers who only ordered in week 1 or week 2, but not both
(
    week1_sales                                           # left table
      .merge( 
          right = week2_sales,                            # right table
          how = 'outer',                                  # full outer join
          left_on = 'Customer ID',                        # on week1_sales.Customer ID = week1_sales.Customer ID
          right_on = 'Customer ID'
      )
      .query(
          "`Food ID_x`.isna() or `Food ID_y`.isna()",     # filter to where left food, x, is null or right food, y, is null
          engine = 'python'             
      )
)

,Customer ID,Food ID_x,Food ID_y
1,97,4.0,NaN
2,658,1.0,NaN
3,202,2.0,NaN
6,213,8.0,NaN
7,600,1.0,NaN
...,...,...,...
449,855,NaN,4.0
450,559,NaN,10.0
451,276,NaN,4.0
452,556,NaN,10.0


In [30]:
# spot check customer 252
week2_sales[week2_sales['Customer ID'] == 252]

,Customer ID,Food ID
248,252,9


In [31]:
week1_sales[week1_sales['Customer ID'] == 252]

,Customer ID,Food ID


In [32]:
# indicator paramter to show you where each row came from
(
    week1_sales
      .merge(
          right = week2_sales,
          how = 'outer', 
          left_on = 'Customer ID', 
          right_on = 'Customer ID', 
          indicator = True                        # indicates which table each row came from            
      )
)

,Customer ID,Food ID_x,Food ID_y,_merge
0,537,9.0,5.0,both
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
4,155,9.0,3.0,both
...,...,...,...,...
449,855,NaN,4.0,right_only
450,559,NaN,10.0,right_only
451,276,NaN,4.0,right_only
452,556,NaN,10.0,right_only


## left join

In [33]:
# customers who ordered in week 1 but not in week 2
(
    week1_sales
      .merge(
          right = week2_sales,
          how = 'left',
          left_on = 'Customer ID',
          right_on = 'Customer ID'
      )
      .query(
          "`Food ID_y`.isna()",
          engine = 'python'
      )
)


,Customer ID,Food ID_x,Food ID_y
1,97,4,NaN
2,658,1,NaN
3,202,2,NaN
5,213,8,NaN
6,600,1,NaN
...,...,...,...
252,413,9,NaN
253,926,6,NaN
254,134,3,NaN
255,396,6,NaN


In [34]:
week1_sales.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [35]:
foods.head()

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99
3,4,Quesadilla,4.25
4,5,Pizza,2.49


In [36]:
# what were the unique foods ordered in week 1?
(
    week1_sales
      .merge(
          right = foods,
          how = 'left',
          left_on = 'Food ID',
          right_on = 'Food ID'
      )
      ['Food Item']
      .unique()
)

array(['Donut', 'Quesadilla', 'Sushi', 'Burrito', 'Salad', 'Pizza',
       'Taco', 'Pasta', 'Steak', 'Drink'], dtype=object)

In [37]:
# how many of each food was ordered in week 1?
(
    week1_sales
      .merge(
          right = foods,
          how = 'left',
          left_on = 'Food ID',
          right_on = 'Food ID'
      )
      ['Food Item']
      .value_counts()
)

Donut         35
Pasta         28
Steak         27
Burrito       26
Sushi         24
Drink         24
Salad         23
Taco          23
Quesadilla    21
Pizza         19
Name: Food Item, dtype: int64

In [38]:
# where there any food items sold in week1 where we don't know what it was?
(
    week1_sales
      .merge(
          right = foods,
          how = 'left',
          left_on = 'Food ID',
          right_on = 'Food ID'
      )
      .query(
          "`Food Item`.isna()",
          engine = 'python'
      )
)

# no

,Customer ID,Food ID,Food Item,Price


## left_on and right_on parameters in .merge() method

In [39]:
customers.head()

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive
3,4,Steven,Evans,Male,Fatz,Registered Nurse
4,5,Judy,Morrison,Female,Demivee,Legal Assistant


In [40]:
week2_sales.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [41]:
# how many of each gender ordered in week 2?
(
    week2_sales                           # left table
      .merge(
          right = customers,              # right table
          how = 'left',                   # left outer join
          left_on = 'Customer ID',        # on Customer ID = ID
          right_on = 'ID'
      )
      [['Customer ID', 'Gender']]         # select these two columns
      .drop_duplicates()                  # SELECT DISTINCT Customer ID, Gender
      [['Gender']]                        # count how many times each gender appeared
      .value_counts()
)

Gender
Female    115
Male      109
dtype: int64

In [42]:
115+109

224

In [43]:
week2_sales['Customer ID'].nunique()

224

In [44]:
# how many times was each food item ordered?
(
    week2_sales                           # left table
      .merge(
          right = customers,              # right table
          how = 'left',                   # left outer join
          left_on = 'Customer ID',        # on Customer ID = ID
          right_on = 'ID'
      )                                   # result of merge is left table
      .merge(
          right = foods,                  # right table
          how = 'left',                   # left outer join
          left_on = 'Food ID',            # on Food ID = Food ID
          right_on = 'Food ID'
      )                                   # one row per order
      ['Food Item']
      .value_counts()
)

Drink         35
Burrito       31
Quesadilla    28
Pasta         26
Salad         25
Steak         23
Taco          23
Pizza         20
Donut         20
Sushi         19
Name: Food Item, dtype: int64

## left_index and right_index parameters
- boolean arguments, True or False, default is False
- can mix and match
  - both columns: left_on and right_on
  - both indices: left_index = True and right_index = True
  - left column, right index: left_on = ..., right_index = True
  - right column, left index: left_index = True, right_on = ...

In [48]:
(
    week1_sales.set_index('Customer ID')        # left table; Customer ID is the index
      .merge(
          right = customers,                    # right table
          how = 'left',                         
          left_index = True,                    # week1_sales.index = customers['ID']
          right_on = 'ID'
      )
)

,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
536,9,537,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
96,4,97,Amanda,Watkins,Female,Ozu,Account Coordinator
657,1,658,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
201,2,202,Louis,Campbell,Male,Rhynoodle,Account Representative III
154,9,155,Carolyn,Diaz,Female,Gigazoom,Database Administrator III
...,...,...,...,...,...,...,...
412,9,413,Diane,Bailey,Female,Wikibox,Technical Writer
925,6,926,Anne,Wagner,Female,Skyba,Legal Assistant
133,3,134,Diana,Hall,Female,Quinu,Financial Advisor
395,6,396,Juan,Romero,Male,Zoonder,Analyst Programmer


## .join() method